In [25]:
import pandas as pd
import json
import os
import requests as r

In [26]:
webform_url = "https://webform.akvo.org/api"
forms = [{"raw_id":611800981, "prod_id":628680982, "instance": "seap"}]

In [27]:
def get_frame(file_id, rename):
    file_json = open(f"./mapping/editor_{file_id}.json", "r")
    res = json.load(file_json)
    file_json.close()
    res = pd.DataFrame(res["questions"], columns=["keyId","variableName","text", "path"])
    res["path"] = res["path"].apply(lambda x: x.split("/")[-1])
    missing_vars = res[res['variableName'] != res['variableName']].to_dict("records")
    for missing_var in missing_vars:
        text = missing_var["text"]
        print(f"- Missing Variables on {rename}: {file_id} {text}")
    rename = {"keyId": f"{rename}_id", "text": f"{rename}_text", "path": f"{rename}_group"}
    rename.update({"variableName": "variable_name"})
    res = res.rename(columns=rename)
    return res

In [28]:
form_maps = []
for form in forms:
    display(HTML("<hr>"))
    instance = form["instance"]
    prod_id = form["prod_id"]
    raw_id = form["raw_id"]
    print(f"Instance Name: {instance}")
    print(f"Raw ID       : {raw_id}")
    print(f"Production ID: {prod_id}")
    raw = get_frame(raw_id, "raw" )
    prod = get_frame(prod_id, "prod")
    merged = raw.merge(prod, on='variable_name')
    webform = r.get(f"{webform_url}/generate/{instance}/{prod_id}")
    webform = r.get(f"{webform_url}/form/{webform.text}")
    webform = webform.json()
    questions = merged.dropna().reset_index()
    questions = questions[["variable_name","raw_id","prod_id"]]
    questions = questions.rename(columns={"raw_id":"id"})
    questions = questions[["id","prod_id"]]
    questions = questions.to_dict("records")
    form_maps.append({
        "id": raw_id,
        "instance": instance,
        "prod_id": prod_id,
        "name": webform["name"],
        "questions": questions
    })

Instance Name: seap
Raw ID       : 611800981
Production ID: 628680982
- Missing Variables on raw: 611800981 Who is the candidate?
- Missing Variables on prod: 628680982 Who is the candidate?


In [29]:
json_object = json.dumps(form_maps, indent = 4)
with open("./forms.json", "w") as outfile:
    outfile.write(json_object)